# Amplitude embedding (simulator)

Main steps:

* initialize an image of the sky (frequency domain)
* apply 2D FT --> visibilities (Fourier domain)
* encode visibilities data into qubits using amplitude embedding (non-conventional domain)
* measure qubits on simulator --> expected outcomes (back to conventional domain)
* apply 2D IFT --> fidelity computation

In [2]:
import numpy as np
import math
import sys

import pennylane as qml
from pennylane import numpy as pnp

## 1) Generating an artificial image of the sky (frequency domain)

In [4]:
#image of the sky filled with double precision complex floats ('complex64')
#pixels are set to low complex random values (image background/noise) 
#few pixels are set to larger complex random values in a specified ellipse area (image source/subject)

n = 4
image = np.zeros((n, n), dtype='complex64')
image.real = np.random.rand(n , n) / 100
image.imag = np.random.rand(n , n) / 100

h, w = image.shape
mask = circular_mask(h, w, radius=h/2)
sky_image = image.copy()
sky_image[~mask] = complex(np.random.rand() * 100, np.random.rand() * 100)
print(sky_image)

[[7.2362640e+01+7.3188263e+01j 7.2362640e+01+7.3188263e+01j
  9.5202513e-03+7.7680424e-03j 7.2362640e+01+7.3188263e+01j]
 [7.2362640e+01+7.3188263e+01j 3.4098811e-03+6.7873118e-03j
  7.2286343e-03+7.8209024e-03j 2.3322478e-03+4.6570646e-03j]
 [5.7000727e-03+2.0873714e-03j 9.6580572e-03+4.4698892e-03j
  3.8421343e-03+6.2812346e-06j 2.3990057e-03+8.6915903e-03j]
 [7.2362640e+01+7.3188263e+01j 9.0144193e-03+5.5422657e-03j
  5.2610035e-03+2.9907073e-03j 9.8879309e-03+4.6035522e-03j]]


## 2) Applying a 2D FT (Fourier domain)

In [5]:
visibilities = np.fft.fft2(sky_image).astype('complex64')
print(visibilities)

[[ 361.88147 +365.99673j   217.06662 +219.54083j    72.35749  +73.17419j
   217.06892 +219.55576j ]
 [ 217.08197 +219.5685j     72.3487   +73.18645j   -72.34708  -73.184555j
    72.34416  +73.17432j ]
 [  72.35663  +73.17889j   -72.365105 -73.19019j  -217.06876 -219.55731j
   -72.350525 -73.17609j ]
 [ 217.06972 +219.54611j    72.36227  +73.1849j    -72.35413  -73.1543j
    72.34992  +73.168j   ]]


#### Sanity check

In [6]:
img = np.fft.ifft2(visibilities).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()
 
print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[7.2362640e+01+7.3188263e+01j 7.2362640e+01+7.3188263e+01j
  9.5214844e-03+7.7676773e-03j 7.2362640e+01+7.3188263e+01j]
 [7.2362640e+01+7.3188263e+01j 3.4108162e-03+6.7868233e-03j
  7.2283745e-03+7.8196526e-03j 2.3331642e-03+4.6563148e-03j]
 [5.7010651e-03+2.0866394e-03j 9.6578598e-03+4.4698715e-03j
  3.8433075e-03+5.7220459e-06j 2.4003983e-03+8.6908340e-03j]
 [7.2362640e+01+7.3188263e+01j 9.0155602e-03+5.5413246e-03j
  5.2609444e-03+2.9902458e-03j 9.8891258e-03+4.6029091e-03j]]

Reals MSE: 6.483969e-13
Imaginaries MSE: 3.3931107e-13


## 3) Data encoding: amplitude embedding (non-conventional domain)

In [9]:
# Amplitude embedding encodes a normalized 2^n-dimensional feature vector into the state of n qbits (uses log2(n) qbits for n classical data)
n = visibilities.shape[0]*visibilities.shape[1]
data = visibilities.flatten()

#normalization to prepare a qstate with measurement probabilites summing up to 1 (SUM (amplitudes²) = 1)
norm = qml.math.sum(qml.math.abs(data) ** 2)
normalized_data = data / qml.math.sqrt(norm)

wires = range(int(math.log2(n))) # set the number of qbits (no padding needed if outputs an integer=integer.0)
amp_dev = qml.device('default.qubit', wires)

@qml.qnode(amp_dev)
def amp_encoding(data):
    qml.AmplitudeEmbedding(data, wires) # normalize = True
    return qml.state() #qml.expval(qml.PauliZ(wires=wires))

results = amp_encoding(normalized_data).astype('complex64')
print(results)

print()

results = results*qml.math.sqrt(norm) # denormalization of the measurements outcomes
results = np.array(results).reshape(sky_image.shape[0] , sky_image.shape[1])
print(results)

[ 0.39311033+0.39758071j  0.23579857+0.23848629j  0.07860164+0.07948881j
  0.23580107+0.2385025j   0.23581524+0.23851635j  0.07859209+0.07950213j
 -0.07859033-0.07950008j  0.07858716+0.07948896j  0.0786007 +0.07949392j
 -0.07860991-0.0795062j  -0.23580088-0.23850419j -0.07859407-0.07949088j
  0.23580192+0.23849203j  0.07860683+0.07950045j -0.07859799-0.07946721j
  0.07859342+0.07948209j]

[[ 361.88147 +365.99673j   217.06664 +219.54083j    72.35749  +73.17419j
   217.06894 +219.55576j ]
 [ 217.08197 +219.56851j    72.3487   +73.18645j   -72.34708  -73.184555j
    72.34416  +73.17432j ]
 [  72.35663  +73.17889j   -72.365105 -73.19019j  -217.06876 -219.55731j
   -72.350525 -73.17609j ]
 [ 217.06972 +219.54611j    72.36227  +73.1849j    -72.35413  -73.1543j
    72.34992  +73.168j   ]]


## 4) Applying 2D IFT (fidelity test)

In [10]:
img = np.fft.ifft2(results).astype('complex64')
print(img)
test_real = ((sky_image.real - img.real)**2).mean()
test_imag = ((sky_image.imag - img.imag)**2).mean()

print()

print('Reals MSE: '+ str(test_real))
print('Imaginaries MSE: ' + str(test_imag))

[[7.2362640e+01+7.3188263e+01j 7.2362640e+01+7.3188263e+01j
  9.5195770e-03+7.7686310e-03j 7.2362640e+01+7.3188263e+01j]
 [7.2362640e+01+7.3188263e+01j 3.4098625e-03+6.7868233e-03j
  7.2255135e-03+7.8196526e-03j 2.3322105e-03+4.6563148e-03j]
 [5.7029724e-03+2.0856857e-03j 9.6578598e-03+4.4689178e-03j
  3.8414001e-03+4.7683716e-06j 2.4003983e-03+8.6898804e-03j]
 [7.2362640e+01+7.3188263e+01j 9.0165138e-03+5.5413246e-03j
  5.2599907e-03+2.9902458e-03j 9.8900795e-03+4.6029091e-03j]]

Reals MSE: 1.9469138e-12
Imaginaries MSE: 8.2619095e-13


## Utils

### Quantum

### Classical

In [3]:
#creates a circular mask over a 2D array
def circular_mask(h, w, center=None, radius=None):
    if center is None: #image center
        center = (int(w/2), int(h/2))
    if radius is None: #smallest distance between center and image bounderies
        radius = min(center[0], center[1], w-center[0], h-center[1])
        
    Y, X = np.ogrid[:h, :w]
    dist_from_center = np.sqrt((X - center[0])**2 + (Y-center[1])**2)
    mask = dist_from_center <= radius
    
    return mask